1)Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

2)Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

3)Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

4)Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

5)Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

6)Могла ли модель переобучиться? Почему?

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]],dtype = np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1],dtype = np.float64)


In [6]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [7]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [8]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [9]:
def calc_logloss(y, y_pred):
    y_pred[y_pred < 0.0001] = 0.01
    y_pred[y_pred > 0.9999] = 0.99
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [10]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
        print(i, W, err)
    return W

In [11]:
alpha_list = [0.1,0.01,0.001,0.0001,0.00001,0.000001,0.0000001]
for i in alpha_list:
    eval_model(X_st, y, iterations=1000, alpha=i)

1000 [-2.77136565 -0.99588853  0.56641089  3.26813012] 0.40587815270727734
1000 [-0.29764618 -0.72670545  1.061634    1.39666497] 0.5233765331724747
1000 [ 0.25805314 -0.68199349  0.68835345  1.24114037] 0.5906912249945011
1000 [ 0.45886878 -0.25439917  0.64531344  1.46695851] 1.0395158244739489
1000 [ 0.49282757 -0.15007512  0.64748969  1.51727928] 1.2013133809011647
1000 [ 0.49632455 -0.13944699  0.64766909  1.52245352] 1.2181844180943702
1000 [ 0.49667518 -0.13838259  0.6476866   1.52297221] 1.2198774477617755


Наилучший результат при alpha = 0.1

In [12]:
W = eval_model(X_st, y, iterations=1000, alpha=0.1)
        

1000 [-2.77136565 -0.99588853  0.56641089  3.26813012] 0.40587815270727734


In [13]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(W,X_st))
    return y_pred_proba

In [14]:
calc_pred_proba(W, X)

array([0.25846265, 0.3056887 , 0.81910473, 0.28147361, 0.79118436,
       0.57257036, 0.93504121, 0.15077287, 0.24742752, 0.85843113])

In [19]:
def calc_pred(W,X):
    y_pred = calc_pred_proba(W, X)
    y_pred[y_pred>0.5] = 1
    y_pred[y_pred<=0.5] = 0
    return y_pred

In [20]:
calc_pred(W,X)

array([0., 0., 1., 0., 1., 1., 1., 0., 0., 1.])

In [23]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [22]:
y_pred = calc_pred(W,X)

Матрица ошибок

In [64]:
TP = np.count_nonzero(np.take(y_pred, np.where(y == 1)))
FN = np.count_nonzero(np.take(y_pred, np.where(y == 1))==0)
FP = np.count_nonzero(np.take(y_pred, np.where(y == 0)))
TN = np.count_nonzero(np.take(y_pred, np.where(y == 0))==0)

In [74]:
import pandas as pd
pred =["a(x) = + 1","a(x) = - 1"]
col1 = [ TP , FN ]
col2 = [ FP , TN ]
df = pd.DataFrame({
    "": pred,
    "y= + 1": col1,
    "y= - 1": col2
})

df

,,y= + 1,y= - 1
0,a(x) = + 1,4,1
1,a(x) = - 1,1,4


In [72]:
accuracy = np.size(np.where(y == y_pred))/len(y)
accuracy

0.8

In [75]:
precision = TP/(TP+FP)
precision

0.8

In [76]:
recall = TP/(TP+FN)
recall

0.8

In [80]:
F1 = 2*precision*recall/(precision+recall)
F1

0.8000000000000002

6)Могла ли модель переобучиться? Почему?

Как мне кажеться в данном случае модель не переобучилась, потому как все метрики одинаковы. Плюс, колчичество объектов класса 1 значимо не прывышают объекты класса 0.